In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df_train = pd.read_csv("train.csv", index_col='id')
df_train.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
id,,,,,,,,,,,,,,,
1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [3]:
df_test = pd.read_csv("test.csv", index_col='id')
df_test.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
id,,,,,,,,,,,,,,
0,0.353600,0.738780,0.600939,0.293377,0.285691,0.458006,0.620704,0.422249,0.369203,0.435727,0.550540,0.699134,0.286864,0.364515
2,0.907222,0.189756,0.215531,0.869915,0.301333,0.528958,0.390351,0.521112,0.794779,0.798580,0.446475,0.449037,0.916964,0.513002
6,0.179287,0.355353,0.623972,0.437812,0.282476,0.320826,0.386789,0.776422,0.222268,0.229102,0.211913,0.222651,0.327164,0.827941
7,0.359385,0.181049,0.551368,0.206386,0.280763,0.482076,0.506677,0.362793,0.379737,0.345686,0.445276,0.518485,0.299028,0.598166
10,0.335791,0.682607,0.676481,0.219465,0.282861,0.581721,0.748639,0.350158,0.448915,0.506878,0.817721,0.805895,0.790591,0.249275


In [4]:
submission = pd.read_csv("sample_submission.csv", index_col='id')
submission

,target
id,
0,0.5
2,0.5
6,0.5
7,0.5
10,0.5
...,...
499984,0.5
499985,0.5
499987,0.5


In [5]:
df_target = df_train.pop('target')

In [6]:
df_target

id
1         7.243043
3         8.203331
4         7.776091
5         6.957716
8         7.951046
            ...   
499995    7.385215
499996    7.242617
499997    9.592487
499998    8.207951
499999    8.890285
Name: target, Length: 300000, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_train, df_target, train_size=0.60, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold


param_grid = {
    'n_estimators':[1000],
    'max_depth' : [12, 14, 16],
    'min_samples_leaf' : [14, 16, 18],
    'min_samples_split' : [6, 8, 10],
    'max_features' : ['auto', 'sqrt'],
}

kf = KFold(random_state=30,
          n_splits=10,
          shuffle=True,
          )

estimator = RandomForestRegressor()

grid_search = GridSearchCV(estimator=estimator,
                          param_grid=param_grid,
                          cv=kf,
                          n_jobs=-1,
                          verbose=2
                          )

grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 126.1min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 529.2min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed: 1112.5min


In [ ]:
grid_search.best_params_

In [ ]:
rf_model = RandomForestRegressor(max_depth=12,
                                 max_features='auto',
                                 min_samples_leaf=14,
                                 min_samples_split=6,
                                 n_estimators=1000)

In [ ]:
rf_model.fit(df_train, df_target)
submission['target'] = rf_model.predict(df_test)
submission.to_csv('random_forest_regressor.csv')

In [ ]:
submission

In [ ]:
preds = rf_model.predict(X_test)

def RMSE(y_test, preds):
    return np.sqrt(mean_squared_error(y_test, preds))
                   
print(RMSE(y_test, preds))